<a href="https://colab.research.google.com/github/KCL-Health-NLP/nlp_examples/blob/master/ann/evaluating_spacy_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data:
https://www.clips.uantwerpen.be/conll2003/ner/



## ***Make sure to change your runtime type to GPU***

In [ ]:
# https://spacy.io/usage
# seems ok to ignore the errors?
!pip install -U pip setuptools wheel
!pip install -U 'spacy[transformers,cuda-autodetect]'

## ***You must retart your runtime now***

In [ ]:
# We are using spaCy for NER.
import spacy

# DocBin is a serialisable collection of spacy
# Documents.
from spacy.tokens import DocBin

# Displacy provides a graphic display of
# documents and annotations, and Scorer scores...
from spacy import displacy
from spacy.scorer import Scorer

# Example holds spacy documents,
# one with predicted annotations
# and one with gold standard .
# We will use it when evalusating.
from spacy.training import Example


In [ ]:
# Check we are using torch.cuda
import torch
print('Torch available:', torch.cuda.is_available())
print('Number of torch devices:', torch.cuda.device_count())
print('Torch current device:', torch.cuda.current_device())

In [ ]:
# Get a locale error on spacy init with GPU - here's a quick fix
# Code from https://github.com/explosion/spaCy/issues/11909
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

The ```en_core_web_lg``` model is a CNN based entity recogniser, trained on severl datasets.

The ```en_core_web_trf``` model is based on the [roBERTa](https://huggingface.co/docs/transformers/model_doc/roberta) pre-trained transformer model, fine-tuned with the same datasets as used for ```en_core_web_lg``` training.

You can find out more about the models and the data used to train them in the [spaCy model documentation](https://spacy.io/models/en).


In [ ]:
# Download spacy models
!python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_trf

[CoNLL 2003 dataset](https://www.clips.uantwerpen.be/conll2003/ner/)

[CoNLL 2003 NER shared task](https://aclanthology.org/W03-0419/)

In [ ]:
# Get the data
!wget https://github.com/KCL-Health-NLP/nlp_examples/raw/master/ann/conll2003_test.txt

In [ ]:
# Take a look at our test data
!head -100 ./conll2003_test.txt

* CoNLL LOC == spaCy GPE + spaCy LOC
* CoNLL PER == spaCy PERSON
* CoNLL ORG == spaCy ORG

Exercise: sed command - use the syntac given to write sed commands

In [ ]:
!sed -i 's/-LOC/-GPE/g' conll2003_test.txt
!sed -i 's/-PER/-PERSON/g' conll2003_test.txt

In [ ]:
# Take another look at our test data
!head -100 ./conll2003_test.txt

In [ ]:
# Convert to .spacy DocBin format
!python -m spacy convert ./conll2003_test.txt . -c ner -n 10


NB if you did not restart your runtime after installing spaCy (first code cell), then downloading the trf model will fail.

In [ ]:
# Load spacy models
nlp_lg = spacy.load('en_core_web_lg')
nlp_tr = spacy.load('en_core_web_trf')

In [ ]:
docs = DocBin().from_disk("./conll2003_test.spacy")

In [ ]:
print(len(docs))

In [ ]:
for doc in docs.get_docs(nlp_lg.vocab):
  print(doc.ents)


Exercise: complete the scoring function given below

In [ ]:
def run_and_score_nlp(docs, pipeline):
  scorer = Scorer()
  examples = []
  for gold_doc in docs.get_docs(pipeline.vocab):
    pred_doc = pipeline(gold_doc.text)
    ex = Example(pred_doc, gold_doc)
    examples.append(ex)
  return (scorer.score(examples), examples)

***Add crude timing in to this next cell***

In [ ]:

scores_lg, examples_lg = run_and_score_nlp(docs, nlp_lg)
scores_tr, examples_tr = run_and_score_nlp(docs, nlp_tr)



Exercise: given that the structure of the scores data structure is X, comlpete the code below to print our the scores

In [ ]:
labels = ['GPE', 'ORG', 'PERSON']
metrics = ['p', 'r', 'f']
print(f'{"label": <18}{"score": <8}{"lg": <6}{"tr": <6}')
for l in labels:
  for m in metrics:
    lg = scores_lg['ents_per_type'][l][m]
    tr = scores_tr['ents_per_type'][l][m]
    print(f'{l: <18}{m: <8}{lg: <6.2f}{tr: <6.2f}')

## Display some documents

In [ ]:
# Add titles to ocuments - displacy will render these titles.

doc_num = 150

doc_tr = examples_tr[doc_num].predicted
doc_tr.user_data["title"] = "Transformer model predictions"

doc_lg = examples_lg[doc_num].predicted
doc_lg.user_data["title"] = "Large model predictions"

doc_ref = examples_tr[doc_num].reference
doc_ref.user_data["title"] = "Reference standard"



Exercise: write code to render all three documents. You will need to make sure you only display the entity labels we are interested in. See the [displacy documentation](https://spacy.io/usage/visualizers#ent) for information on how to do this. Put a line of dashes or some newlines between each document.

In [ ]:
# Display in displacy
displacy.render(doc_tr, style='ent', jupyter=True, options={'ents':labels})
print('\n'*2)
displacy.render(doc_lg, style='ent', jupyter=True, options={'ents':labels})
print('\n'*2)
displacy.render(doc_ref, style='ent', jupyter=True, options={'ents':labels})